In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Reactive
using Interact
using SplineUtils
using SplineRaceWay
using HierarchicalRecedingHorizonController
using NearestNeighbors

In [ ]:
# scene parameters
scene = Scene()
framerate = 24
Δt = 1.0/framerate # size of rendering timesteps
n_integration_sub_steps = 3 # sub steps for smoother integration
context = IntegratedContinuous(Δt, n_integration_sub_steps) # integrated continuous context

#car parameters
car_length = 4.8 # front wheel to back wheel
car_width = 2.5
v⁰  = 0.0 # initial velocity
δ⁰ = 0.0 # initical steering angle

############### INITIALIZE TRACK ################
lane_width = 30.0
radius = 45.0
edge_buffer = 0.25
T_MAX = lane_width/2.0 - car_width/2.0 - edge_buffer # max allowable projection distance from center of lane
base_speed= 0.0

# spline control points
Pts = 50*[0 -1 -2 -3 -3.5 -3 -2 -1 -0.5 -1 -2 -3 -4 -5 -6 -6.5 -6 -5.5 -6 -6 -5 -4 -3 -2 -1.5 -1 0 1 1.5 2 3 4 5 6.25 7 7 7 7 7 7 7 6 5 4 4 4 3 2 1 0; 
       0 0 0 0 -1 -2 -2 -2 -3 -4 -4 -4 -4 -4 -4 -3 -2 -1 0 1 2 3 4 4 3 2 2 2 3 4 4 4 4 4 3 2 1 0 -1 -2 -3 -4 -4 -3 -2 -1 0 0 0 0]
degree = 3 # degree of spline
num_points = 10001
num_samples = 420

track = Raceway(Pts,degree,num_points,num_samples,lane_width)
track;

In [ ]:
carcolors = Dict{Int, Colorant}()
colors = [colorant"yellow",colorant"green",colorant"blue",colorant"red"]
numCars = 4
curve_idxs = Set()
while length(curve_idxs) < 12
    push!(curve_idxs,rand(1:num_samples))
end
idx_list = [i for i in curve_idxs]
for i in 1:numCars
    roadind = RoadIndex(CurveIndex(idx_list[i],0),LaneTag(1,1))
    vehstate = VehicleState(Frenet(roadind, track.roadway), track.roadway, 0.0)
    vehdef = VehicleDef(i,AgentClass.CAR, car_length, car_width)
    push!(scene,Vehicle(vehstate, vehdef))
    k = rand(1:4)
    carcolors[i] = colors[k]
    track.models[i] = mapHRHC(i,track.roadway,context,h=12,v_max=100.0+rand(1:20),
    μ=30.0 + rand(1:5),a_step=10.0 + rand(1:3),a_range=[-1,0,1],k=rand(1:4))
end

actions = Array(DriveAction, length(scene))

track.obstacleMap = generateObstacleMap(scene, track.models)
track;

In [ ]:
function drawsim(t, dt=NaN)
    k_level = 0 # needs to be updated into a loop
    for k_level in 0:maximum([model.k for (id,model) in track.models])
        for (i, veh) in enumerate(scene)
            model = track.models[veh.def.id]
            observe!(model, scene, track.roadway, veh.def.id, track.tree, track.obstacleMap, k_level)
            actions[i] = rand(model)
        end
    end
    for (veh, action) in zip(scene, actions)
        model = track.models[veh.def.id]
        context = action_context(model)
        veh.state = propagate(veh, action, context, track.roadway)
    end
    render(scene, track.roadway, cam=FitToContentCamera(), car_colors=carcolors)
#     zoom = 3.0
#     render(scene, track.roadway, cam=SceneFollowCamera(zoom), car_colors=carcolors)
end

In [ ]:
ticks = fps(framerate)
timestamps = map(_ -> time(), ticks)
map(drawsim, timestamps)

In [ ]:
# using Reel
# film = roll(drawsim, fps=framerate, duration=20)
# write("Level3Overtake.gif", film) # Write to a gif file